In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


In [ ]:
predictions_IRV2      = np.load("predictions_IRV2.npy")
predictions_Densenet  = np.load("predictions_Densenet.npy")
predictions_Resnet50  = np.load("predictions_Resnet50.npy")
predictions_Resnet34  = np.load("predictions_Resnet34.npy")
predictions_VGG16     = np.load("predictions_VGG16.npy")

y_true = np.load("y_true.npy")


In [ ]:
import numpy as np

predictions = np.stack([
    predictions_IRV2,
    predictions_Densenet,
    predictions_Resnet50,
    predictions_Resnet34,
    predictions_VGG16
], axis=0)

print(predictions.shape)


In [ ]:
def prob_to_belief(prob):
    """
    prob: (num_classes,)
    returns belief mass (same shape)
    """
    prob = np.clip(prob, 1e-8, 1.0)
    return prob / np.sum(prob)


In [ ]:
def ds_combine(belief1, belief2):
    """
    belief1, belief2: (num_classes,)
    """
    num_classes = belief1.shape[0]
    
    combined = np.zeros(num_classes)
    conflict = 0.0

    for c in range(num_classes):
        combined[c] = belief1[c] * belief2[c]

    # Conflict = sum of mass assigned to empty intersections
    conflict = np.sum(belief1 * (1 - belief2))

    # Normalize
    combined = combined / (1.0 - conflict + 1e-8)

    return combined, conflict


In [ ]:

def ds_fuse_models(model_probs):
    """
    model_probs: (num_models, num_classes)
    """
    belief = prob_to_belief(model_probs[0])

    total_conflict = 0.0

    for i in range(1, model_probs.shape[0]):
        belief_i = prob_to_belief(model_probs[i])
        belief, conflict = ds_combine(belief, belief_i)
        total_conflict += conflict

    return belief, total_conflict


In [ ]:
num_samples = predictions.shape[1]
num_classes = predictions.shape[2]

teacher_probs = np.zeros((num_samples, num_classes))
conflicts = np.zeros(num_samples)

for i in range(num_samples):
    fused, conflict = ds_fuse_models(predictions[:, i, :])
    teacher_probs[i] = fused
    conflicts[i] = conflict

print("Teacher predictions shape:", teacher_probs.shape)
print("Mean conflict:", np.mean(conflicts))


In [ ]:
y_true = test_batches.classes
y_pred = np.argmax(teacher_probs, axis=1)

from sklearn.metrics import accuracy_score, classification_report

print("Teacher Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=targetnames))
